# Description 
Write the simulator for a queue with k servers, and waiting line of size N, which implements a strict priority service discipline (with preemption).

Customers are partitioned into two classes: High Priority (HP) customers and Low Priority (LP)

Both arrival processes are Poisson with rate  lambda_{HP}  and  lambda_{LP} respectively. 

LP customers are served only when no  HP customers are in the waiting line.

The service of a LP customer is  potentially interrupted  upon the arrival of   a HP customer if no  servers are idle.  Furthermore, upon the arrival of a HP customer, to accomodate the arriving customers,  a LP customer is potentially  dropped if there is not room in the waiting line.

Plot of the average delay  for HP customers, LP customers, end the aggregate average delay 

(i.e. the average delay on all customers) when lambda_{HP}=\lambda_{LP}=0.2, 0.4, 0.8, 1.4, 2.0, 2.4, 

2.8, k=2, N=1000

a) E[S]_{HP}= E[S]_{LP}=1.0 

b) E[S]_{HP}=1/2 and  E[S]_{LP}=3/2  

Consider the three "usual" distributions for the service time:

EXP: exponentially distributed with mean= E[S]_{*P}

DET: deterministic =E[S]_{*P}

HYP:  distributed according to a hyper-exponential distribution with mean=E[S]_{*P} standard 

deviation=10*E[S]_{*P}

In [1]:
import random


class Server:
    def __init__(self, server_name, server_status=0):
        self.server_name = server_name
        self.server_status = server_status


class PriorityQueue:
    def __init__(self):
        self.elements = []
        self.arrival_time_list = []
        self.departure_time_list = []

    def empty(self):
        return len(self.elements) == 0

    def put(self, item, priority, arrival_time):
        self.elements.append([priority, item])
        inter_arrival = random.expovariate(1.0 / 2)
        self.arrival_time_list.append(arrival_time + inter_arrival)
        self.departure_time_list.append(0)

    def get(self):
        # Get the highest priority element
        highest_priority_element = max(self.elements, key=lambda x: x[0])
        highest_priority_index = self.elements.index(highest_priority_element)
        if self.departure_time_list[highest_priority_index] == 0:
            self.departure_time_list = [self.arrival_time_list[highest_priority_index] + service_time for _ in
                                        self.departure_time_list]
        else:
            self.departure_time_list = [element + service_time for element in self.departure_time_list]
        dep_time = self.departure_time_list[highest_priority_index]
        arr_time = self.arrival_time_list[highest_priority_index]
        # Remove the highest priority element
        self.elements.remove(highest_priority_element)
        del self.departure_time_list[highest_priority_index]
        del self.arrival_time_list[highest_priority_index]
        # Return the highest priority element
        return highest_priority_element, arr_time, dep_time


S1 = Server(server_name='S1')
S2 = Server(server_name='S2')
pq = PriorityQueue()
service_time = 4 # this is where we can deal wit the time distros 
arrive_time = 0
counter = 0
c1 = 0 #client 1 arrival time 
c2 = 0 # client 2  arrival time 
for i in range(100):
    arrive_time += 1
    counter += 1
    priority = random.randint(0, 1)
    pq.put('client' + str(counter), priority, arrive_time)
# priority = 0 => LP
# priority =1 => HP
    arrive_time += 1
    counter += 1
    priority = random.randint(0, 1)
    pq.put('client' + str(counter), priority, arrive_time)
    # arrive_time += 1
    print(pq.elements)
    print(pq.arrival_time_list)
    print('===========')
    if c1 > arrive_time:
        S1.server_status = 1 #1 = working , 0 = idle
    else:
        S1.server_status = 0
    if c2 > arrive_time:
        S2.server_status = 1
    else:
        S2.server_status = 0
    if S1.server_status == 0:
        a, b, c1 = pq.get()
        print(S1.server_name, a, b, c1, sep=' - ')
    if S2.server_status == 0:
        a, b, c2 = pq.get()
        print(S2.server_name, a, b, c2, sep=' - ')
    input()

for i in range(5):
    pass


[[0, 'client1'], [0, 'client2']]
[1.5237239232397082, 2.0451648076553335]
S1 - [0, 'client1'] - 1.5237239232397082 - 5.523723923239708
S2 - [0, 'client2'] - 2.0451648076553335 - 9.523723923239707

[[1, 'client3'], [0, 'client4']]
[5.036134407084921, 4.349726668078236]

[[1, 'client3'], [0, 'client4'], [1, 'client5'], [1, 'client6']]
[5.036134407084921, 4.349726668078236, 6.81492194860886, 6.240273167347282]
S1 - [1, 'client3'] - 5.036134407084921 - 9.036134407084921

[[0, 'client4'], [1, 'client5'], [1, 'client6'], [0, 'client7'], [0, 'client8']]
[4.349726668078236, 6.81492194860886, 6.240273167347282, 8.274307667414655, 8.675818560586404]

[[0, 'client4'], [1, 'client5'], [1, 'client6'], [0, 'client7'], [0, 'client8'], [1, 'client9'], [1, 'client10']]
[4.349726668078236, 6.81492194860886, 6.240273167347282, 8.274307667414655, 8.675818560586404, 9.422398221813689, 10.005463019242715]
S1 - [1, 'client5'] - 6.81492194860886 - 13.036134407084921
S2 - [1, 'client6'] - 6.240273167347282 - 1

KeyboardInterrupt: Interrupted by user